In [16]:

def score_wrapper(scoring_function, step, total_steps, model=None):
    try:
        if model:
            scoring_function(model)
        else:
            scoring_function()
    except:
        print(f'ERROR step {step}')
    return f'data {step/total_steps} \n\n'

In [17]:
def test():
    print('in test')
def test2():
    print('in test2')

In [6]:
score_wrapper(test, 0, 10)

<generator object score_wrapper at 0x7f99c47290b0>

In [20]:
def generate():
    yield score_wrapper(test, 1, 10)
    yield score_wrapper(test2, 2, 10)

for i in generate():
    print(i)
    

in test
data 0.1 


in test2
data 0.2 




In [3]:
import subprocess
import os

os.system('cd data')
os.system('pwd')

FileNotFoundError: [Errno 2] No such file or directory: 'cd'